# Proyecto 2

* Flavio Galán
* Josue Say
* Juan Pablo Solís
* Isabella Miralles

[Link Kaggle](https://www.kaggle.com/competitions/lmsys-chatbot-arena)

## Situación Problemática

## Problema Científico

## Objetivos

## Análisis Exploratorio

### Técnicas comunes para detección de patrones en texto

### Preprocesamiento de los datos

### EDA

## Conclusiones